In [1]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from threading import Lock
import traceback
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

counter_lock = Lock()

def scrape(accr_code):
    global counter
    driver = None
    search_results_data = []
    transfer_history_data = []
    try:
        opts = FirefoxOptions()
        opts.add_argument("--headless")
        driver = webdriver.Firefox(options=opts)
        driver.get("https://www.rec-registry.gov.au/rec-registry/app/public/stc-register")

        # Making sure the search box is ready and visible
        search_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'accreditationCode')))
        search_box.send_keys(accr_code, Keys.ENTER)
        #time.sleep(10)

        # Making sure the table is ready and visible
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-ranges-results"]/tbody/tr')))
        s_rows = driver.find_elements(By.XPATH, '//*[@id="search-ranges-results"]/tbody/tr')

        # Check if large amount of data is returned
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-ranges-results_info"]')))
        search_results_range = driver.find_element(By.XPATH, '//*[@id="search-ranges-results_info"]').text
        exceed_10_rows_flag = int(re.findall(r'\d+', search_results_range)[-1]) > 10

        if exceed_10_rows_flag:
            raise Exception('Large amount of data returned')

        for s_row in s_rows:
            s_cols = s_row.find_elements(By.TAG_NAME, 'td')
            s_cols_data = [ele.text for ele in s_cols]  
            search_results_data.append(s_cols_data)

            if s_cols_data == ['No data available in table']:
                raise Exception('Accreditation code not found')
                
            # Open the popup window
            link = s_cols[7].find_element(By.TAG_NAME, 'a')
            link.click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="transfer-history-results"]/tbody/tr')))
            t_rows = driver.find_elements(By.XPATH, '//*[@id="transfer-history-results"]/tbody/tr')
            
            for t_row in t_rows:
                t_cols = t_row.find_elements(By.TAG_NAME, 'td')
                t_cols_data = [ele.text for ele in t_cols]
                if t_cols_data == ['No data available in table']:
                    t_cols_data.append("None")
                    t_cols_data.append("None")
                else:
                    t_cols_data.append(s_cols[7].text)
                    t_cols_data.append(s_cols[1].text)

                transfer_history_data.append(t_cols_data)

            # Close the popup windows (ready for the next s_row (search results row))
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="certificate-details-dialog"]')))
            html = driver.find_element(By.XPATH, '//*[@id="certificate-details-dialog"]')
            html.send_keys(Keys.ESCAPE)

    except Exception as e:
        if str(e) == 'Accreditation code not found':
            print(f"Accreditation code {accr_code} not found.")
            transfer_history_data.append(['None','None','None','None',accr_code])
            if driver:
                driver.quit()
        elif str(e) == 'Large amount of data returned':
            print(f"Accreditation code {accr_code} contains a large ammount of data.")
            transfer_history_data.append(['None','None','None','None',accr_code])
            if driver:
                driver.quit()
        else:
            print(f"An error occurred processing: {accr_code} in scrape: {e}")
            traceback.print_exc()
            if driver:
                driver.quit()
    finally:
       if driver:
           driver.quit()
    with counter_lock:
        counter += 1
        print(f"Processed {counter} out of {len(accreditation_Code_List)}")
    
    return search_results_data, transfer_history_data

def scrape_all(accreditation_Code_List):
    search_results_all = []
    transfer_history_all = []
    
    with ThreadPoolExecutor(max_workers=50) as executor:
        results = executor.map(scrape, accreditation_Code_List)

    for result in results:
        search_results, transfer_history = result
        search_results_all.extend(search_results)
        transfer_history_all.extend(transfer_history)   
    return search_results_all, transfer_history_all

if __name__ == '__main__':
    
    base_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+'/'
    accreditation_Code_List = pd.read_pickle(base_dir+'0_datasets/accreditation_code_list.pkl')
    accreditation_Code_List = accreditation_Code_List.tolist()
    accreditation_Code_List = accreditation_Code_List[200:220]
   
    s_headers = "Current owner","Accreditation code","Fuel source","Generation year","Creation year","Generation state","Status","Certificate serial number","Certificate quantity"
    t_headers = "Transferred date","Seller","Buyer","Certificate Number","accreditation_Code"

    counter = 0
    search_results_data, transfer_history_data = scrape_all(accreditation_Code_List)

    search_results_data = pd.DataFrame(search_results_data, columns=s_headers)
    transfer_history_data = pd.DataFrame(transfer_history_data, columns=t_headers)

Accreditation code BEBGNS20 not found.
Accreditation code HYMINS02 not found.
Accreditation code BEBGNS20 not found.
Processed 1 out of 20
Processed 2 out of 20
Accreditation code BEBGNS20 not found.
Processed 3 out of 20
Processed 4 out of 20
Processed 5 out of 20
Processed 6 out of 20
Processed 7 out of 20
Processed 8 out of 20
Processed 9 out of 20
Processed 10 out of 20
Processed 11 out of 20
Accreditation code BEBGNS20 not found.
Accreditation code BEBGNS20 not found.
Processed 12 out of 20
Processed 13 out of 20
Accreditation code BEBGNS20 not found.
Processed 14 out of 20
Processed 15 out of 20
Processed 16 out of 20
Processed 17 out of 20
Processed 18 out of 20
Processed 19 out of 20
Processed 20 out of 20


In [2]:
search_results_data

,Current owner,Accreditation code,Fuel source,Generation year,Creation year,Generation state,Status,Certificate serial number,Certificate quantity
0,Go Markets Environmental Trading Pty Ltd,PVD2394811,S.G.U. - solar (deemed),2015,2016,QLD,Invalid due to audit,1-41,41
1,Vic Solar Technologies Pty Ltd,PVD2394786,S.G.U. - solar (deemed),2016,2016,WA,Invalid due to audit,1-26,26
2,Vic Solar Technologies Pty Ltd,PVD2394788,S.G.U. - solar (deemed),2016,2016,WA,Invalid due to audit,1-26,26
3,P and N NSW Pty Ltd,PVD2394840,S.G.U. - solar (deemed),2016,2016,NSW,Invalid due to audit,1-101,101
4,AICA Engineering Pty Ltd,PVD2394850,S.G.U. - solar (deemed),2016,2016,WA,Invalid due to audit,1-47,47
5,Wallandale Pty Ltd,SW2394790,S.W.H. - solar (deemed),2015,2016,QLD,Invalid due to audit,1-28,28
6,Emerging Energy Solutions Group Pty Ltd,PVD2394872,S.G.U. - solar (deemed),2016,2016,NSW,Invalid due to audit,1-103,103
7,SolarpowerRex Pty Ltd,PVD2394912,S.G.U. - solar (deemed),2015,2016,QLD,Invalid due to audit,1-62,62
8,FORMBAY TRADING PTY LTD,PVD2394881,S.G.U. - solar (deemed),2016,2016,VIC,Invalid due to audit,1-9,9
9,No data available in table,None,None,None,None,None,None,None,None


In [3]:
transfer_history_data

,Transferred date,Seller,Buyer,Certificate Number,accreditation_Code
0,No data available in table,None,None,None,None
1,No data available in table,None,None,None,None
2,No data available in table,None,None,None,None
3,No data available in table,None,None,None,None
4,No data available in table,None,None,None,None
5,No data available in table,None,None,None,None
6,No data available in table,None,None,None,None
7,No data available in table,None,None,None,None
8,No data available in table,None,None,None,None
9,None,None,None,None,HYMINS02
